# Fabrício Ferreira da Silva RA: 231900 e Leandro Stival RA: 263013

## Descrição
Notebook contendo a criação de um SVM para classificar imagens de resonância em EM ou AVC, treinamento realizando com o conjunto gerado através das imagens fornecidas, que foram normalizadas utilizando uma quantização para 30 tons de cinza e posterioremente tiveram suas *features* extraídas de forma manual e salvas em arquivos *.csv* que foram carregados unidos, normalizados e separados em treino e validação.
<br><br>
O treinamento foi realizado utilizando o ajuste fino de parâmetros para criar o melhor modelo possível, e a sua avaliação com os dados de validação foi realizado com a acurácia balanceada junto de uma matriz de confusão.

In [1]:
#Lib para importar o SVM
from sklearn.svm import SVC
#Lib para separar treino e validação
from sklearn.model_selection import train_test_split

#Libs para tratamento dos dados
import pandas as pd
import numpy as np
import os

## Lendo os dados

In [2]:
def normalize_df(df):
    """
    Recebe um conjunto de dados e retorna o seu valor normalizado
    """
    df_norm = (df-df.min())/(df.max()-df.min())
    return df_norm

### Conjunto de dados EM

In [3]:
#Conjunto de dados de EM
EM_data = pd.read_csv('EM_metrics_mask.csv', index_col=0)
EM_data['class'] = 1
EM_data.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation,SRE,LRE,GLU,RLU,RPC,class
0,1288.867496,12.202771,0.799151,0.361724,0.601435,0.718118,1.630,5629.021,7530.316,16462.360,23.716,1
1,1266.846531,11.826299,0.807538,0.365931,0.604923,0.715535,1.611,5870.977,7759.470,15719.644,23.033,1
2,1029.659424,10.246641,0.830054,0.443281,0.665793,0.681310,1.743,7440.006,8320.351,16857.419,21.516,1
3,983.164949,9.845900,0.835705,0.453394,0.673345,0.685854,1.717,7771.060,8569.443,16014.342,20.925,1
4,919.637582,9.422576,0.839422,0.463695,0.680952,0.694344,1.710,8207.277,8896.347,15309.346,20.185,1


### Conjunto de dados AVC

In [4]:
#Conjunto de dados com AVC
AVC_data = pd.read_csv('AVC_metrics_mask.csv', index_col=0)
AVC_data.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation,SRE,LRE,GLU,RLU,RPC,class
0,1625.604039,19.820663,0.636765,0.207224,0.455219,0.883667,1.689,1798.881,3501.078,28358.894,39.208,0
1,1553.799974,19.881742,0.631335,0.211914,0.460341,0.892043,1.691,1833.233,3492.157,28625.825,39.316,0
2,1329.088520,18.463883,0.642487,0.224320,0.473625,0.905728,1.626,2016.729,3647.280,26321.974,37.666,0
3,1146.232647,16.944070,0.659995,0.242467,0.492409,0.912077,1.551,2338.977,3928.393,23145.349,34.988,0
4,1002.614520,16.302926,0.646247,0.193333,0.439696,0.903753,1.450,1876.947,3869.643,21385.778,35.493,0


### Unindo conjunto de dados

Os dados são após lidos são unidos em um único conjunto de dados *DataFrame* contendo $1048$ amostras com $11$ características extraídas (*features*) e a classe de cada amostra.
<br>
<br>
A normalização dos dados foi realizada após a unificação, para que a escala possa representar ambas as classes e não ficar somente focada em uma (normalizar os dados após a unificação apresentou ganho de 1% em acurácia nos dados de validação).

In [5]:
#Dados de ambas as classes
df_both_class = pd.concat([EM_data, AVC_data])

In [6]:
#Normalizando os dados unidos
df_both_class = normalize_df(df_both_class)

In [7]:
#Separando os valores em X e y para criar treino e validação

#Treinamento
X = df_both_class.iloc[:,:-1]

#Rótulos
y = df_both_class.iloc[:,-1:].values.ravel()

## Preparando os dados

Os dados foram separados em conjunto de treinamento e validação em uma proporção de 70/30, assim 70% dos dados foram alocados para serem utilizadas para treinar a nosso classificador baseado em SVM, enquanto 30% foi utilizado para validar a capacidade de generalização do modelo.
<br>
<br>
Para que o conjunto gerado seja sempre o mesmo foi utilizado uma semente fixa de 2022, garantindo a replicabilidade do experimento, outro ponto a se atentar é a estratificação dos conjuntos, assim tornando a distribuição das amostras positivas e negativas (AVC e EM) igualitária entre treino e validação.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=2022,stratify=y)

In [9]:
import warnings
warnings.filterwarnings('always')

## Preparando o modelo

Visando gerar um modelo robusto foi realizado a otimização dos parâmetros de treinamento através da técnica de busca em grande (*Grid Search*), essa técnica treina o modelo com diversas combinações de parâmetros e seleciona aqueles que apresentaram uma qualidade melhor (conforme o teste com a métrica selecionada).
<br>
<br>
Nosso modelo durante o treinamento foi avaliado através da sua precisão, revocação e acurácia balanceada, junto do método *classification_report* que permite uma visualização direta desses valores ao final do treinamento, além da evolução da qualidade conforme os parâmetros foram testados.

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [11]:
tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000]},
]

In [12]:
scores = ["precision", "recall", "balanced_accuracy"]

## Treinando

O treinamento basicamente ocorre uma vez para cada método de avaliação definido na lista *scores* e através do *GridSearch* busca a melhor configuração dos parâmetros para o SVM, ao final do treinamento o resultado é apresentado para cada um dos *scores* demonstrando a qualidade por interação do *GridSeach*.
<br>
<br>
Como resultado do treinamento algumas combinações apresentaram 100% de acurácia (ou outra métrica que estava sendo utilizada na interação) no conjunto de treino, mostrando assim que é possível separar os conjuntos através de uma representação linear (considerando que durante o ajuste fino dos parâmetros foi selecionado o *Kernel* linear).

In [13]:
#Exemplo de fine tunning obtido do tutorial do SKlearn para finetunning

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}

Grid scores on development set:

0.513 (+/-0.013) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.513 (+/-0.013) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.959 (+/-0.047) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.513 (+/-0.013) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.994 (+/-0.024) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.959 (+/-0.047) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.994 (+/-0.024) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.994 (+/-0.024) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
1.000 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 100, 'gamma': 0.00

C:\Users\stiva\.conda\envs\mo826\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stiva\.conda\envs\mo826\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stiva\.conda\envs\mo826\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}

Grid scores on development set:

0.500 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.500 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.977 (+/-0.026) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.500 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.994 (+/-0.016) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.977 (+/-0.026) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.997 (+/-0.013) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.994 (+/-0.016) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
1.000 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
1.000 (+/-0.000) for {'C': 100, 'gamma': 0.001, 'kernel': 'linear'}
1.000 (+/-0.000) f

## Testando qualidade com a validação

Em posse do modelo treinado foi obtida a matriz de confusão e a acurácia balanceada do classificador, os resultados foram bem promissores com os dados de validação com uma acurácia de 99% e errando somente 1 amostra de validação.

In [14]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [15]:
y_pred = clf.predict(X_test)

In [16]:
#Acurácia balanceada para os dados de validação
balanced_accuracy_score(y_test, y_pred)

0.9986033519553073

In [17]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [18]:
#Matriz de confusão:
tn, fp, fn, tp
print(f"Verdadeiro Negativo: {tn} \nFalso Negativo: {fp}\nFalso Positivo: {fn}\nVerdadeiro Positivo: {tp}")

Verdadeiro Negativo: 357 
Falso Negativo: 1
Falso Positivo: 0
Verdadeiro Positivo: 376
